# Vaccination status
Adults 
- Pneumococcal
- Flu vaccine (time-sensitive: within current flu season)
- Shingles


Children 
See NHS website


Vaccinations are recorded in two places:
1. A vaccination table (not in the current build of dummy data, needs adding in later)
2. Read codes in the `CodedEvent` table


In [1]:
import pyodbc
import pandas as pd
from IPython.display import display, Markdown

### Server connection

In [2]:
server = 'covid.ebmdatalab.net,1433'
database = 'OPENCoronaExport' 
username = 'SA'
password = 'ahsjdkaJAMSHDA123[' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

### Flu codes
- Haphazardly taken from here, for speed: https://clinicalcodes.rss.mhs.man.ac.uk/medcodes/article/6/codelist/influenza-immunisation/
- Looks like there's a completely different set in `../data/QoFClusteres_CTV3Codes - Sheet1.csv`

In [3]:
flu_read_codes = pd.read_csv('../data/flu_codes.csv')
flu_read_codes.head()

,Code,Coding system,Description,Entity type,List name,QOF,analysis
0,65E..,Read,Influenza vaccination,immunisation,Influenza immunisation,1,Main
1,9OX8.,Read,Has influenza vaccination at work,immunisation,Influenza immunisation,0,Sensitivity
2,n473.,Read,NaN,immunisation,Influenza immunisation,1,Main
3,n477.,Read,NaN,immunisation,Influenza immunisation,1,Main
4,n47d.,Read,NaN,immunisation,Influenza immunisation,1,Main


In [4]:
def codes_to_sql_where(col_name, code_list):
    where = ""
    i = 0
    for code in code_list:
        if i == 0:
            where = where + f"{col_name} = '{code}'"
        else:
            where = where + f" OR {col_name} = '{code}'"
        i+=1
    return where

In [5]:
codes_to_sql_where("CTV3Code", flu_read_codes['Code'].head())

"CTV3Code = '65E..' OR CTV3Code = '9OX8.' OR CTV3Code = 'n473.' OR CTV3Code = 'n477.' OR CTV3Code = 'n47d.'"

### Medcode based vaccinations
- there are no values for the current flu season in the dummy data, but it works for previous time periods

In [6]:
# NBVAL_IGNORE_OUTPUT
codes_where = codes_to_sql_where("CTV3Code", flu_read_codes['Code'])
query = f'''
SELECT * --DISTINCT Patient_ID
FROM CodedEvent
WHERE ({codes_where}) AND ConsultationDate BETWEEN '2019-09-01' AND '2020-03-31'
'''
df = pd.read_sql(query, cnxn, index_col='Patient_ID')
df

,Consultation_ID,CodedEvent_ID,CTV3Code,NumericValue,ConsultationDate,SnomedConceptId
Patient_ID,,,,,,


### Connection should be closed before restarting the kernal or closing the notebook

In [7]:
cnxn.close()